<a href="https://colab.research.google.com/github/wangwangwang77/Master_thesis/blob/main/master_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import math
import random
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib._color_data as mcd
import matplotlib.patches as mpatch
from scipy.stats import norm
import scipy.stats as ss
from functools import reduce
from time import time
from scipy import optimize

##Heston Model
The Heston Model is a stochastic volatility model whose time evolution is given by the following system of stochastic differential equations:
$$dS_{t}=rS_{t}dt+\sigma_{t}S_{t}dZ_{t} $$ 
$$d\sigma_{t}^{2}=\kappa(\theta-\sigma_{t}^{2})dt+\nu\sqrt{\sigma_{t}^{2}}dB_{t}$$
$\rho\neq0$ denotes the correlation parameter between Brownian motion $Z$ and $B$. In our simulation, we assume $r=0$. 
We rewrite $Z_{t}$ as:
$$Z_{t}=\sqrt{1-\rho^{2}}B_{t}+\rho W_{t}$$
where $B_{t}$ and $Z_{t}$ are independent brownian motion and the Heston Model can be written as:
$$dS_{t}=S_{t}\sigma_{t}(\sqrt{1-\rho^{2}}B_{t}+\rho W_{t})$$

#Crude Monte Carlo
We implement discretization schemes for both the stock and the volatility. Thess dynamics looks as follows:
\begin{align*}
\sigma_{i}^{2}&=\kappa(\theta-\sigma_{i}^{2})\Delta t+\xi\sqrt{\sigma_{i-1}^{2}}\Delta W\\
S_{i}&=S_{i-1}+\sqrt{\sigma_{i-1}^{2}}S_{i-1}(\sqrt{1-\rho^{2}}\Delta B+\rho\Delta W)
\end{align*}
We want to find the call option price:
$$E[\max(S_{T}-K)]$$






In [ ]:
from scipy.stats import norm
def cmcOptionPrice(T, m, F0, K,sigma_0,rho= -0.9, gamma=0.3, theta=0.06, ka=1,n=5000,r=0):
    delta = T/n
    deltaW1 = np.random.normal(0,np.sqrt(delta),(m,n))
    deltaW2 = np.random.normal(0,np.sqrt(delta),(m,n))
    V = np.zeros((m,n+1))
    V[:,0] = sigma_0
    S = np.zeros((m,n+1))
    S[:,0] = F0
    for i in range(n):
       V[:,i+1] = np.maximum(V[:,i]+ka*(theta-V[:,i])*delta+gamma*np.sqrt(V[:,i])*deltaW2[:,i],0)
       S[:,i+1] = S[:,i]+ np.sqrt(V[:,i])*S[:,i]*(np.sqrt(1-rho**2)*deltaW1[:,i]+rho*deltaW2[:,i])
    mc = S[:,n]
    mc = np.maximum(mc-K,0)
    price = np.mean(mc)
    price_std = np.std(mc)/np.sqrt(m)
    lo = price - norm.ppf(0.975)*price_std
    hi = price + norm.ppf(0.975)*price_std
    return (price,lo,hi,price_std)

In [ ]:
m = 2000
T = 1
F0 = 100
K = 100
sigma_0 = 0.04
t1 = time()
cmcOptionPrice(T, m, F0, K,sigma_0 )
print('Time to get call option price: {} seconds'.format(round((time() - t1), 5)))


Time to get call option price: 1.82003 seconds


##Willard's Formula
Let us price a call option:
$$V_{T}=E\lbrack(S_{T}-K)_{+}\rbrack$$
Notice that
\begin{align*}
S_{T}&=S_{0}\exp\left(-\frac{1}{2}\int_{0}^{T}\sigma_{s}^{2}ds+\int_{0}^{T}\sigma_{s}(\sqrt{1-\rho^{2}}B_{s}+\rho W_{s})\right)\\&=S_{0}\exp\left(-\frac{1}{2}(\bar{\sigma_{p}^{2}}T+\rho^{2}\int_{0}^{T}\sigma_{s}^{2}ds)\right)\times\exp(\sqrt{1-\rho^{2}}\int_{0}^{T}\sigma_{s}dB_{s}+\rho\int_{0}^{T}\sigma_{s}dW_{s})
\end{align*}
where $\bar{\sigma_{\rho}}^{2}=\frac{1}{T}\left(\int_{0}^{T}\sigma_{s}^{2}ds-\rho^{2}\int_{0}^{T}\sigma_{s}^{2}ds\right)$.We assume that $\xi=\exp(\rho\int_{0}^{T}\sigma_{s}dW_{s}-\frac{1}{2}\rho^{2}\int_{0}^{T}\sigma_{s}^{2}ds)$. Then we can write $S_{T}$ as 
$$S_{T}=S_{0}\xi\exp(-\frac{1}{2}\bar{\sigma_{\rho}}^{2}T+\sqrt{1-\rho^{2}}\int_{0}^{T}\sigma_{s}dB_{s} )$$
Note that $\sqrt{1-\rho^{2}}\int_{0}^{T}\sigma_{s}dB_{s}=N(0,\bar{\sigma_{p}}^{2}T)$
$$S_{T}=S_{0}\xi\exp(-\frac{1}{2}\bar{\sigma_{p}}^{2}T+\bar{\sigma_{p}}\sqrt{T}N(0,1))$$
That is, $S_{T}$ can be seen (conditioned to $W$) as a process with deterministric volatility given by $\bar{\sigma_{\rho}}$ 
and the initial asset price is $S_{0}\xi$
By the Law of Iterated Expectations, we can re-write $V_{T}$ as 
\begin{align*}
V_{T}&=E\lbrack E\lbrack (S_{T}-K)_{+}|W_{t}\rbrack\rbrack\\
&=E\lbrack BS(S_{0}\xi,K,\bar{\sigma_{p}}^{2},T)\rbrack
\end{align*}


To implement the Monte Carlo simulation, the discretization for the volatility is the first step and given by 
$$\sigma_{t_{i}}^{2}=\max({\sigma_{t_{i-1}}^{2}}+\kappa(\theta-\sigma_{t_{i-1}}^{2})\Delta t+\xi\sqrt{\sigma_{t_{i-1}}^{2}}\Delta W_{i-1},0)$$

We take $S_{0}=K=100,T=1$ and $\kappa=1,\theta=0.09, \sigma_{0}=0.2,\xi=0.3,\rho=-0.5,n=5000$ in the simulation. 



##Improved conditional Monte Carlo (O(max(m,n))) 

In [ ]:
def BlackScholes(F0, K, sigma, T, r=0):
    d1 = (np.log(F0 / K) + sigma**2 / 2 * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    BSprice = np.exp(-r * T) * (F0 *  ss.norm.cdf(d1) - K * ss.norm.cdf(d2))
    return(BSprice)

def mcOptionPrice(Z,T, m, F0, K,sigma_0,rho=-0.9, gamma=0.3, theta=0.1, ka=1,n=5000,r=0):
        OptionPrice = 0
        volatility_swap = 0
        variance_swap = 0
        adjustment = 0
        sigmasquare = np.zeros((m,n+1))
        sigmasquare[:,0] = sigma_0
        sigmasquare_integral = np.array([])
        for j in range(0,n):
             sigmasquare[:,j+1] = np.maximum(sigmasquare[:,j]+ka*(theta-sigmasquare[:,j])*T/n+gamma*np.sqrt(sigmasquare[:,j])*Z[:,j],0)
        # stochastic volatility compute from the GBM from 0 to T
        # integral approximation as finite sums
        for i in range(0,m):
             deterministic_integral = np.sum(sigmasquare[i,0:-1])*T/n       # 1/n := constant time step, then 1/n = t(i+1)-t(i) for all i 
             sigmasquare_integral = np.append(sigmasquare_integral,deterministic_integral)
             stochastic_integral    = np.dot(np.sqrt(sigmasquare[i,0:-1]),Z[i,:])   
             ksi_0 = np.exp(rho*stochastic_integral-0.5*rho**2*deterministic_integral)    
             sigma_rho = np.sqrt((1/T)*(1-rho**2)*deterministic_integral)
             OptionPrice +=BlackScholes(F0 * ksi_0,K,sigma_rho,T)/m    
             volatility_swap+= np.sqrt(deterministic_integral/T)/m
             variance_swap+=deterministic_integral/(T*m)
        adjustment = np.var(sigmasquare_integral)/8/(math.pow(variance_swap,3/2))
        return  OptionPrice,volatility_swap,variance_swap, adjustment

In [ ]:
 n = 5000
 T = 0.0001
 m = 10000
 Z = np.random.normal(0,np.sqrt(T/n),[m,n])
 F0 = 100
 K = 100
 sigma_0 = 0.05
 t2 = time()
 mcprice, vol,var2,ad = mcOptionPrice(Z,T, m, F0, K,sigma_0)
 vol
 print('Time to get call option price: {} seconds'.format(round((time() - t2), 5)))

Time to get call option price: 8.40826 seconds


In [ ]:
 mcprice, vol,var2,ad 

(0.08829510520723309,
 0.22361791915871315,
 0.05000573349596793,
 1.6985102389823407e-14)

In [ ]:
np.sqrt(var2)-vol 

1.698710488856614e-06

We calculate the $\E_{t}(\sigma)=\sqrt{\E_{t}(\frac{1}{T}\int_{0}^{T}\sigma_{t}^{2})}$ below

Next, we use brent function find the implied volatility

In [ ]:
vol

0.22362156481031045

In [ ]:
def nrImplied_Volatility(mcOptionPrice,F0,T,K):
    
    def f(I0):
        f=(BlackScholes(F0,K,I0,T,r=0)-mcOptionPrice)**2
        return(f) 
    
    I =optimize.brent(f,brack=(0.001,3))
    return(I)

In [ ]:
 I1 = nrImplied_Volatility(mcprice,F0,T,K)
 I1

0.21924696653987175

In [ ]:
I1-vol


-0.0043745982704387065

Comparing the implied volatilty we obtained with volatility's mean, the value is close but not equal.

#Anithetic conditional Monte Carlo

In [ ]:
from scipy.optimize.nonlin import Jacobian
def BlackScholes(F0, K, sigma, T, r=0):
    d1 = (np.log(F0 / K) + sigma**2 / 2 * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    BSprice = np.exp(-r * T) * (F0 *  ss.norm.cdf(d1) - K * ss.norm.cdf(d2))
    return(BSprice)

def mcOptionPrice1(Z,T, m, F0, K,sigma_0,rho= -0.9, gamma=0.3, theta=0.06, ka=1,n=5000,r=0):
        OptionPrice = 0
        volatility_swap = 0
        variance_swap2 = 0
        sigmasquare = np.zeros((m,n+1))
        sigmasquare_2 = np.zeros((m,n+1))
        sigmasquare[:,0] = sigma_0
        sigmasquare_2[:,0] = sigma_0
        deltaW2 = -Z
        adjustment = 0
        sigmasquare_integral = np.array([])
        for j in range(0,n):
            sigmasquare[:,j+1] = np.maximum(sigmasquare[:,j]+ka*(theta-sigmasquare[:,j])*T/n+gamma*np.sqrt(sigmasquare[:,j])*Z[:,j],0)
            sigmasquare_2[:,j+1] = np.maximum(sigmasquare_2[:,j]+ka*(theta-sigmasquare_2[:,j])*T/n+gamma*np.sqrt(sigmasquare_2[:,j])*deltaW2[:,j],0)
        # stochastic volatility compute from the GBM from 0 to T
        # integral approximation as finite sums 
        for i in range(0,m):
            deterministic_integral = np.sum(sigmasquare[i,0:-1])*T/n       # 1/n := constant time step, then 1/n = t(i+1)-t(i) for all i 
            sigmasquare_integral = np.append(sigmasquare_integral,deterministic_integral)
            stochastic_integral    = np.dot(np.sqrt(sigmasquare[i,0:-1]),Z[i,:])   
            ksi_0 = np.exp(rho*stochastic_integral-0.5*rho**2*deterministic_integral)    
            sigma_rho = np.sqrt((1/T)*(1-rho**2)*deterministic_integral)
            deterministic_integral_2 = np.sum(sigmasquare_2[i,0:-1])*T/n       # 1/n := constant time step, then 1/n = t(i+1)-t(i) for all i 
            sigmasquare_integral = np.append(sigmasquare_integral,deterministic_integral_2)
            stochastic_integral_2    = np.dot(np.sqrt(sigmasquare_2[i,0:-1]),deltaW2[i,:])   
            ksi_0_2 = np.exp(rho*stochastic_integral_2-0.5*rho**2*deterministic_integral_2)    
            sigma_rho_2 = np.sqrt((1/T)*(1-rho**2)*deterministic_integral_2)
            OptionPrice +=(BlackScholes(F0 * ksi_0,K,sigma_rho,T)+BlackScholes(F0 * ksi_0_2,K,sigma_rho_2,T))/(2*m)   
            volatility_swap+= np.sqrt(deterministic_integral/T)/(2*m)+np.sqrt(deterministic_integral_2/T)/(2*m)
            variance_swap2+=(deterministic_integral/T+deterministic_integral_2/T)/(2*m) 
        adjustment = np.var(sigmasquare_integral)/8/(math.pow(variance_swap2,3/2))
        return  OptionPrice,volatility_swap,variance_swap2,adjustment


In [ ]:
 m = 10000
 T = 0.0001
 n = 5000
 Z = np.random.normal(0,np.sqrt(T/n),(m,n))
 F0 = 100
 K = 100
 sigma_0 = 0.05
 t2 = time()
 mccall_price,vol,var,ad= mcOptionPrice1(Z,T, m, F0, K,sigma_0)
 #print('Time to get call option price: {} seconds'.format(round((time() - t2), 5)))

In [ ]:
mccall_price,vol,var,ad

(0.08912624312485887,
 0.22360624084256508,
 0.050000498698076504,
 1.6720253887067148e-14)

In [ ]:
np.sqrt(var)-vol

1.6720274326476048e-06

In [ ]:
vol

0.22360112837174068

In [ ]:
def nrImplied_Volatility(mcOptionPrice,F0,T,K):
    
    def f(I0):
        f=(BlackScholes(F0,K,I0,T,r=0)-mcOptionPrice)**2
        return(f) 
    
    I =optimize.brent(f,brack=(0.001,3))
    return(I)

In [ ]:
 I1 = nrImplied_Volatility(mccall_price,F0,T,K)
 I1 - vol

-0.0026124001650709994

In [ ]:
I1

0.22194940564740437

In [ ]:
rho= -0.9;
ka=1;
sigma_0 = 0.05;
#T = [0.5,0.4,0.3,0.2,0.1,0.01,0.001,0.0001];
a= rho**2*sigma_0**2/98/sigma_0+rho*sigma_0/8;
for i in T:
  print (a*i);



-0.002605867346938776
-0.002084693877551021
-0.0015635204081632654
-0.0010423469387755104
-0.0005211734693877552
-5.211734693877552e-05
-5.2117346938775515e-06
-5.211734693877552e-07


Conditional Monte Carlo of SABR

In [ ]:
def mcOptionPrice_MC_SABR(Z, alpha, sigma_0, rho, T, n, F0, K,m):
    OptionPrice = 0
    volatility_swap = 0
    variance_swap = 0
    adjustment = 0
    sigmasquare_integral = np.array([])
    for i in range(0,m):
        Z1 = Z[i,]
        t = np.arange(0,T+T/n,T/n)
        Z2 = Z1*np.sqrt(T/n)
        W2 = np.zeros(n+1)
        W2[1:n+1] = np.cumsum(Z2)
        sigma = sigma_0*np.exp(-0.5 * alpha**2 * t[0:n+1] + alpha * W2) # stochastic volatility compute from the GBM from 0 to T
        # integral approximation as finite sums
        deterministic_integral = np.sum(sigma[0:-1]**2)* T / n       # 1/n := constant time step, then 1/n = t(i+1)-t(i) for all i
        sigmasquare_integral = np.append(sigmasquare_integral,deterministic_integral)
        stochastic_integral    = np.dot(sigma[0:-1],np.diff(W2))   
        ksi_0 = np.exp(rho * stochastic_integral - 0.5 * rho**2 * deterministic_integral)    
        sigma_rho = np.sqrt(1/T * (1-rho**2) * deterministic_integral)
        volatility_swap += np.sqrt(deterministic_integral/T)/m
        OptionPrice += BlackScholes(F0 * ksi_0, K, sigma_rho, T)/m    
        variance_swap +=deterministic_integral/(T*m)
    adjustment = np.var(sigmasquare_integral)/8/(math.pow(variance_swap,3/2))
    return(OptionPrice,volatility_swap,variance_swap,adjustment)

Anithetic conditional Monte Carlo of SABR

In [ ]:
def mcOptionPrice_AMC_SABR(Z, alpha, sigma_0, rho, T, n, F0, K,m):
    OptionPrice = 0
    volatility_swap_SABR = 0
    variance_swap_2 = 0
    adjustment = 0
    sigmasquare_integral = np.array([])
    for i in range(0,m):
        Z1 = Z[i,]
        t = np.arange(0,T+T/n,T/n)
        Z2 = Z1*np.sqrt(T/n)
        W2 = np.zeros(n+1)
        delta_W = np.zeros(n+1)
        W2[1:n+1] = np.cumsum(Z2)
        delta_W  = -W2
        sigma = sigma_0*np.exp(-0.5 * alpha**2 * t[0:n+1] + alpha * W2) # stochastic volatility compute from the GBM from 0 to T
        # integral approximation as finite sums
        sigma_2 = sigma_0*np.exp(-0.5 * alpha**2 * t[0:n+1] + alpha * delta_W)
        deterministic_integral_1 = np.sum(sigma[0:-1]**2)* T / n       # 1/n := constant time step, then 1/n = t(i+1)-t(i) for all i
        sigmasquare_integral = np.append(sigmasquare_integral,deterministic_integral_1)
        stochastic_integral_1   = np.dot(sigma[0:-1],np.diff(W2))   
        deterministic_integral_2= np.sum(sigma_2[0:-1]**2)* T / n       # 1/n := constant time step, then 1/n = t(i+1)-t(i) for all i
        sigmasquare_integral = np.append(sigmasquare_integral,deterministic_integral_2)
        stochastic_integral_2   = np.dot(sigma_2[0:-1],np.diff(delta_W)) 
        ksi_0_1 = np.exp(rho * stochastic_integral_1 - 0.5 * rho**2 * deterministic_integral_1) 
        ksi_0_2 = np.exp(rho * stochastic_integral_2 - 0.5 * rho**2 * deterministic_integral_2)   
        sigma_rho_1 = np.sqrt(1/T * (1-rho**2) * deterministic_integral_1)
        sigma_rho_2 = np.sqrt(1/T * (1-rho**2) * deterministic_integral_2)
        volatility_swap_SABR += np.sqrt(deterministic_integral_1/T)/(2*m)+np.sqrt(deterministic_integral_2/T)/(2*m)
        OptionPrice += BlackScholes(F0 * ksi_0_1, K, sigma_rho_1, T)/(2*m)+ BlackScholes(F0 * ksi_0_2, K, sigma_rho_2, T)/(2*m)
        variance_swap_2+=(deterministic_integral_1/T+deterministic_integral_2/T)/(2*m)
    adjustment = np.var(sigmasquare_integral)/8/(math.pow(variance_swap_2,3/2))
    return(OptionPrice,volatility_swap_SABR,variance_swap_2,adjustment)

In [ ]:
m = 10000
n = 10000
Z = np.random.normal(0,1,[m,n])
rho = -0.6
alpha = 0.6
sigma_0 = 0.3 # different from Heston model, here sigma value is not sigma square 
K = 100
F0 = 100
T = 0.0001
mccall_price_SABR_1, vol_1,var_1,ad_1= mcOptionPrice_MC_SABR(Z, alpha, sigma_0, rho, T, n, F0, K,m)
mccall_price_SABR_2, vol_2,var_2,ad_2= mcOptionPrice_AMC_SABR(Z, alpha, sigma_0, rho, T, n, F0, K,m)


In [ ]:
mccall_price_SABR_1, vol_1,var_1,ad_1

(0.11891988072956475,
 0.300005193292287,
 0.09000420350750689,
 1.8127558602155507e-14)

In [ ]:
mccall_price_SABR_2, vol_2,var_2,ad_2

(0.11986061498014909,
 0.30000094055484505,
 0.09000165156809069,
 1.8121207631465115e-14)

In [ ]:
np.sqrt(var_1)

0.3000070057640436

In [ ]:
np.sqrt(var_2)

0.30000275260085646

implied volatility of SABR model

In [ ]:
I_1 = nrImplied_Volatility(mccall_price_SABR_1,F0,T,K)
I_2=nrImplied_Volatility(mccall_price_SABR_2,F0,T,K)
I_1-vol_1,I_2-vol_2


(-0.0019171474774493658, 0.0004451789713685872)

In [ ]:
I_1,I_2

(0.2978257354356695, 0.2993216146142442)

In [ ]:
0.01296*0.0001


1.296e-06